In [5]:
import tkinter as tk
root = tk.Tk()
root.title("Entry app")
button = tk.Button(root, text='Stop', width=25, command=root.destroy)
button.pack()
root.mainloop()

In [63]:
#entry app
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
import cv2,numpy as np,pytesseract,imutils,pymongo,datetime
def select_image():
    img = filedialog.askopenfilename()
    if len(img) > 0:
        img = cv2.imread(img,cv2.IMREAD_COLOR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(gray, (3,3), 0)
        edged = cv2.Canny(gray, 30, 200)
        #countour detection
        contours=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours,key=cv2.contourArea, reverse = True)[:10]
        screenCnt = None
        contours1=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                break
        if screenCnt is None:
            detected = 0
            #print ("No contour detected")
        else:
            detected = 1

        if detected == 1:
            cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)
        mask = np.zeros(gray.shape,np.uint8)
        new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
        new_image = cv2.bitwise_and(img,img,mask=mask)
        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        Cropped = gray[topx:bottomx+1, topy:bottomy+1]
        pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\Arjun\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
        text = pytesseract.image_to_string(Cropped, config='--psm 10')
        plate=""
        for x in text.upper():
            if (ord(x) >= ord('A') and ord(x)<=ord('Z')) or (ord(x) >=ord('0') and ord(x)<=ord('9')):
                plate+=x
        myclient = pymongo.MongoClient("mongodb://localhost:27017/") #connecting to the mongo db server using the respective connection string
        mydb = myclient["Parking-DB"]#creating a database named Parking-DB
        mycol = mydb["Index"]#Creating a collection named Index in the above DB i.e. it is used for referencing the available DB for parking slots
        myquery = { "Parking_count": { "$lt": 5 } } 
        mydoc = mycol.find_one(myquery)
        val=mydoc["Parking_count"]
        newvalues = { "$set": {"Parking_count": val+1 } }
        mycol.update_one(myquery, newvalues)
        update_col = mydb[str(mydoc["_id"])]
        myquery1 = {"Reg_No":""}
        mydoc1 = update_col.find_one(myquery1)
        a=datetime.datetime.now()
        newvalues1 ={"$set":{"Reg_No":plate,"Entry_time":str(a)}} #replace PY01BG4545 with variable plate
        update_col.update_one(myquery1,newvalues1)


def dbreset():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/") #connecting to the mongo db server using the respective connection string
    mydb = myclient["Parking-DB"]#creating a database named Parking-DB
    mycol = mydb["Index"]
    #reset
    myquery = { "Parking_count": { "$lte": 6 } }
    newvalues = { "$set": {"Parking_count": 0 } } 
    mycol.update_many(myquery, newvalues)
    basements=['B1','B2','B3']
    for x in basements:
        base_data=x
        col = mydb[x]
        col.update_many({},{"$set":{"Reg_No":"","Entry_time":""}})
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert img into grayscale
root = Tk()
# create a button, then when pressed, will trigger a file chooser
# dialog and allow the user to select an input image; then add the
# button the GUI
root.title("Entry app")
btn = Button(root, text="Select an image", command=select_image)
btn.pack(side="top")#, fill="both", expand="yes", padx="10", pady="10")
btn1=Button(root, text="Reset DB", command=dbreset)
btn1.pack(side="bottom")
button = tk.Button(root, text='Stop', width=25, command=root.destroy)
button.pack(side="right")
# kick off the GUI
root.mainloop()

In [64]:
#exit app
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
import cv2,numpy as np,pytesseract,imutils,pymongo,datetime
def select_image():
    img = filedialog.askopenfilename()
    if len(img) > 0:
        img = cv2.imread(img,cv2.IMREAD_COLOR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(gray, (3,3), 0)
        edged = cv2.Canny(gray, 30, 200)
        #countour detection
        contours=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours,key=cv2.contourArea, reverse = True)[:10]
        screenCnt = None
        contours1=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                break
        if screenCnt is None:
            detected = 0
            #print ("No contour detected")
        else:
            detected = 1

        if detected == 1:
            cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)
        mask = np.zeros(gray.shape,np.uint8)
        new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
        new_image = cv2.bitwise_and(img,img,mask=mask)
        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        Cropped = gray[topx:bottomx+1, topy:bottomy+1]
        pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\Arjun\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
        text = pytesseract.image_to_string(Cropped, config='--psm 10')
        plate=""
        for x in text.upper():
            if (ord(x) >= ord('A') and ord(x)<=ord('Z')) or (ord(x) >=ord('0') and ord(x)<=ord('9')):
                plate+=x
        myclient = pymongo.MongoClient("mongodb://localhost:27017/") #connecting to the mongo db server using the respective connection string
        mydb = myclient["Parking-DB"]#creating a database named Parking-DB
        mycol = mydb["Index"]#Creating a collection named Index in the above DB i.e. it is used for referencing the available DB for parking slots
        search_string=plate #replace PY01BG4545 with variable plate
        basements=['B1','B2','B3']
        base_data=None
        mydoc2=None
        for x in basements:
            base_data=x
            col = mydb[x]
            mydoc2 = col.find_one({"Reg_No":search_string})
            if mydoc2 != None:
                break
        # deletion from db
        # first insert the data of that car into history db
        hist_db = myclient["History_DB"]
        hist_col = hist_db["History_data"]
        a1 = hist_col.find_one({"_id":search_string})
        b=datetime.datetime.now()
        if a1==None:
            hist_col.insert_one({"_id":search_string,"Time":[]})
        hist_col.update_one({"_id":search_string},{"$push":{"Time":(str(mydoc2["Entry_time"]),str(b))}})
        mydb[base_data].update_one({"Reg_No":search_string},{"$set":{"Reg_No":"","Entry_time":""}})
        #mydoc2["Reg_No"] = mydoc2["Entry_time"] =""
        col1 = mydb["Index"]
        col2 = col1.find_one({"_id":base_data})
        val=col2["Parking_count"]
        col1.update_one({"_id":base_data},{"$set":{"Parking_count":val-1}})
root = Tk()
# create a button, then when pressed, will trigger a file chooser
# dialog and allow the user to select an input image; then add the
# button the GUI
root.title("Exit app")
btn = Button(root, text="Select an image", command=select_image)
btn.pack(side="top")#, fill="both", expand="yes", padx="10", pady="10")
button = tk.Button(root, text='Stop', width=25, command=root.destroy)
button.pack(side="right")
# kick off the GUI
root.mainloop()


In [55]:
#search app
import tkinter as tk,qrcode,pymongo
from PIL import Image, ImageTk
# Top level window
frame = tk.Tk()
frame.title("TextBox Input")
frame.geometry('1280x720')
# Function for getting Input
# from textbox and printing it
# at label widget

def printInput():
	inp = inputtxt.get(1.0, "end-1c")
	myclient = pymongo.MongoClient("mongodb://localhost:27017/")
	mydb = myclient["Parking-DB"]#creating a database named Parking-DB
	mycol = mydb["Index"]
	search_string=inp.strip() #replace PY01BG4545 with variable plate
	basements=['B1','B2','B3']
	base_data=None
	mydoc2=None
	for x in basements:
		base_data=x
		col = mydb[x]
		mydoc2 = col.find_one({"Reg_No":search_string})
		if mydoc2 != None:
			break
	text="Basement:"+str(base_data)+"\n Parking no.:"+str(mydoc2["_id"])+"\n Register no:"+str(mydoc2["Reg_No"])+"\n Entry log:"+str(mydoc2["Entry_time"])
	img=qrcode.make(text)
	#img = Image.fromarray(img)
	#img = ImageTk.PhotoImage(img)
	lbl = tk.Label(frame, image =img)
	lbl.pack()
	#lbl.config(image = img)
# TextBox Creation
inputtxt = tk.Text(frame,
				height = 1,
				width = 20)

inputtxt.pack()

# Button Creation
printButton = tk.Button(frame,
						text = "Show QR code",
						command = printInput)
printButton.pack()

# Label Creation
frame.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Arjun\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Arjun\AppData\Local\Temp\ipykernel_12996\3183115338.py", line 31, in printInput
    lbl = tk.Label(frame, image =img)
  File "C:\Users\Arjun\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 3177, in __init__
    Widget.__init__(self, master, 'label', cnf, kw)
  File "C:\Users\Arjun\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 2601, in __init__
    self.tk.call(
_tkinter.TclError: image "<qrcode.image.pil.PilImage object at 0x000001FE92FE7850>" doesn't exist
